In [1]:
from lib.diffusion import StableDiffusionModel
from lib.hair import HairMaskPipeline
from PIL import Image

In [2]:
hair_mask_pipeline = HairMaskPipeline()

input_image_path ='source_data/stefan_photo.jpg'

hair_mask_path = hair_mask_pipeline.generate_hair_mask(image_path=input_image_path, output_mask_path='output/masks/hair_mask.png')

print('Hair mask generated at', hair_mask_path)

prompt = 'Change the lightness of the hair to 50% and color it red. Do not change the face or facial features.'

hair_mask_pipeline.apply_hair_color(input_image_path, hair_mask_path, prompt,
                              output_path='output/colored_hair.jpg')

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

I0000 00:00:1734226187.683162 8020769 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1734226187.685792 8021249 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Hair mask saved to: output/masks/hair_mask.png
Hair mask generated at output/masks/hair_mask.png
Preprocessed image saved to: source_data/stefan_photo.jpg


  0%|          | 0/25 [00:00<?, ?it/s]

Modified image saved to: output/colored_hair.jpg


'output/colored_hair.jpg'